In [1]:
%matplotlib inline
import tensorflow as tf
import tensorflow.keras as K
from tensorflow.keras.datasets import mnist
from sklearn.preprocessing import normalize
import numpy as np
import matplotlib.pyplot as plt
from sklearn import manifold

# Afficher les GPUs

In [2]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
tf.debugging.set_log_device_placement(True)
logical_gpus = tf.config.experimental.list_logical_devices('GPU')
print("Num logical GPUs: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1
Num logical GPUs:  1


# Recuperation des data

In [3]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [4]:
print(x_train.shape)

(60000, 28, 28)


In [5]:
n_classes = 10

x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.

x_train = x_train.reshape((x_train.shape[0],x_train.shape[1],x_train.shape[2], 1))
x_test = x_test.reshape((x_test.shape[0],x_test.shape[1], x_test.shape[2], 1))

y_train = K.utils.to_categorical(y_train, n_classes)
y_test = K.utils.to_categorical(y_test, n_classes)
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)


(60000, 28, 28, 1)
(60000, 10)
(10000, 28, 28, 1)
(10000, 10)


# Construction d'un model d'autoencoder sequentiel test

In [ ]:
encoding_dim = 10
input_shape = x_train.shape[1:]

encoder = K.Sequential([
    K.layers.InputLayer(input_shape),
    K.layers.Conv2D(64, (3,3), activation='relu'),
    K.layers.Conv2D(32, (3,3), activation='relu'),
    K.layers.Conv2D(10, (3,3), activation='relu'),
    K.layers.Flatten(),
    K.layers.Dense(encoding_dim, activation='sigmoid')

])

encoder.compile(optimizer='Adam' ,loss = 'categorical_crossentropy', metrics=['accuracy'])
encoder.summary()


In [ ]:

history = encoder.fit(x_train, y_train,
                epochs=50,
                batch_size=256,
                shuffle=True,
                validation_data=(x_test, y_test))


# learning losses

In [ ]:
ax1, = plt.plot(history.history["loss"],label="train loss")
ax2, = plt.plot(history.history["val_loss"],label="test loss")
plt.legend(handles=[ax1, ax2])
plt.show()
ax1, = plt.plot(history.history["accuracy"],label="accuracy")
ax2, = plt.plot(history.history["val_accuracy"],label="test accuracy")
plt.legend(handles=[ax1, ax2])
plt.show()

print("max accuracy reached is {}%".format(np.max(history.history["val_accuracy"])*100))

# Save model

In [ ]:
model_path = "models/encoder_test.h5"
encoder.save(model_path)
del encoder

# Load model

In [ ]:
encoder = K.models.load_model(model_path)

# Look at predictions

In [ ]:
preds = encoder.predict_classes(x_test)
print(preds.shape)

In [ ]:
n=5
plt.figure(figsize=(20,10))
for i in range(n*n):
    # Display image
    ax = plt.subplot(n, n, i+1)
    plt.imshow(x_test[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    ax.set_title("Predicted {}".format(preds[np.random.randint(preds.shape[0])]))

plt.show()
print("accuracy: {}".format(np.nonzero(preds == y_test)[0].shape[0]/y_test.shape[0]))

# Looking at the latent space

In [ ]:
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
import cv2

# Scatter with images instead of points
def imscatter(x, y, ax, imageData, zoom):
    images = []
    for i in range(len(x)):
        x0, y0 = x[i], y[i]
        # Convert to image
        img = imageData[i]*255.
        img = img.astype(np.uint8).reshape([28,28])
        img = cv2.cvtColor(img,cv2.COLOR_GRAY2RGB)
        # Note: OpenCV uses BGR and plt uses RGB
        image = OffsetImage(img, zoom=zoom)
        ab = AnnotationBbox(image, (x0, y0), xycoords='data', frameon=False)
        images.append(ax.add_artist(ab))
    
    ax.update_datalim(np.column_stack([x, y]))
    ax.autoscale()


# Show dataset images with T-sne projection of latent space encoding
def computeTSNEProjectionOfLatentSpace(X, encoder, display=True):
    # Compute latent space representation
    print("Computing latent space projection...")
    X_encoded = encoder.predict(X)

    # Compute t-SNE embedding of latent space
    print("Computing t-SNE embedding...")
    tsne = manifold.TSNE(n_components=2, init='pca', random_state=0)
    X_tsne = tsne.fit_transform(X_encoded)
    if display:
        # Plot images according to t-sne embedding
        print("Plotting t-SNE visualization...")
        fig, ax = plt.subplots()
        imscatter(X_tsne[:, 0], X_tsne[:, 1], imageData=X, ax=ax, zoom=0.6)
        plt.show()
    return X_tsne
    
X_tsne = computeTSNEProjectionOfLatentSpace(x_test[:100], encoder, display=True)

# Loading Darters